In [2]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import math
import operator


In [9]:
# read saved prices
prices = pd.read_csv("./prices.csv")

In [10]:
prices.head()

,shop,timestamp,product,price
0,CZC,2019-12-16 17:44:28.432887,Mys,12
1,CZC,2019-12-16 17:44:28.446934,Notebook,122
2,CZC,2019-12-16 17:44:28.458476,Mobil,45
3,Alza,2019-12-16 17:44:28.470659,Mys,45
4,Alza,2019-12-16 17:44:28.481968,Notebook,150


In [12]:
class Downloader:
    
    def is_new_product_price(self, product): # checked if the price is different or not
        
        latest_price = 0
        
        for index, row in prices.iterrows():
            if(row["shop"] == self.name and row["product"] == product.get("name")):
                latest_price = row["price"]

        if(latest_price != product.get("price")):
            return True
        
    def update_prices(self): # if the price is different, add a new row
        products = self.get_products()
        
        for product in products:
            if(self.is_new_product_price(product)):
                print("added row")
                
             # how to add a row
                global prices
                prices = prices.append({"shop":self.name, "timestamp": datetime.now(), "product": product.get("name"), "price": product.get("price")}, ignore_index=True)

                
        
class Downloader_CZC(Downloader):
     
    def get_products(self): #here would be the downloader for CZC
        
        return   [
            {"name":"Mys", "price": 23},
            {"name":"Notebook", "price": 178},
            {"name":"Mobil", "price": 336}
        ]
    
    name = "CZC"


class Downloader_Alza(Downloader): #here would be the downloader for Alza
    
    name = "Alza"
    

    def get_products(self):

        return [
            {"name":"Mys", "price": 25},
            {"name":"Notebook", "price": 37},
            {"name":"Mobil", "price": 690}
        ]
    
class Downloader_Mall(Downloader): #here would be the downloader for Mall
    
    name = "Mall"
    

    def get_products(self):

        return [
            {"name":"Mys", "price": 43},
            {"name":"Notebook", "price": 289},
            {"name":"Mobil", "price": 45}
        ]

In [13]:
# add Mall
downloaders = [Downloader_CZC(), Downloader_Alza(), Downloader_Mall()]

In [14]:
# for every downloader in downloader, it downloads the prices, look if it is changed and eventually it adds a new price to prices

for downloader in downloaders:
    downloader.update_prices()
    

added row
added row


In [15]:
prices.head() 

,shop,timestamp,product,price
0,CZC,2019-12-16 17:44:28.432887,Mys,12
1,CZC,2019-12-16 17:44:28.446934,Notebook,122
2,CZC,2019-12-16 17:44:28.458476,Mobil,45
3,Alza,2019-12-16 17:44:28.470659,Mys,45
4,Alza,2019-12-16 17:44:28.481968,Notebook,150


In [16]:
prices.to_csv("prices.csv", index=False) # index = False - deletes the new column which shows which every new price
